In [2]:

# Install dependencies
%pip install --quiet pyreadstat statsmodels seaborn SyntheticControlMethods zstd 


import os
install = '"git+https://github.com/microsoft/SparseSC.git"'
os.system(f'pip install -Uqq {install}')




[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


0

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pyreadstat
from SyntheticControlMethods import Synth
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize, basinhopping
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from statsmodels.stats.sandwich_covariance import cov_hc1
from sklearn.linear_model import LogisticRegression
import warnings
import pyreadstat


ModuleNotFoundError: No module named 'SparseSC'

In [31]:
dat = pd.read_stata('replication/CountryEPData.dta')
westminster = pd.read_csv('replication/westminster.csv')
datIp = pyreadstat.read_dta('replication/CountryEPData_covIP.dta')
# remove france
dat = dat[dat['country'] != 'France']
# remove 2019 election
dat = dat[dat['ep_election'] != 2019]

# merge dat with data_ip on country and ep_election
dat = dat.merge(datIp[0], on=['ccode', 'year', 'ep_election'], how='left')
sub_dat = dat[[
    "rr_votes", "E_eubadIP", "E_NatDem_satisfiedIP",
    "unempIP", "opencIP", "S_uegenIP", "rightgovIP",
    "ch_imp_fns", "eu_pos_meanIP", "migr_inflIP",
    "ccode", "country", "ep_election"
]]

In [32]:
sub_dat

,rr_votes,E_eubadIP,E_NatDem_satisfiedIP,unempIP,opencIP,S_uegenIP,rightgovIP,ch_imp_fns,eu_pos_meanIP,migr_inflIP,ccode,country,ep_election
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Belgium,NaN
1,NaN,0.034266,0.533128,4.2,90.84992,12.8,1.0,NaN,6.427861,NaN,1,Belgium,1979.0
2,NaN,0.038414,0.532901,5.5,96.10307,13.0,1.0,NaN,6.427861,NaN,1,Belgium,1979.0
3,NaN,0.046106,0.531076,6.3,94.97614,13.1,0.0,NaN,6.427861,NaN,1,Belgium,1979.0
4,NaN,0.045521,0.487478,6.8,91.63636,13.2,0.0,NaN,6.427861,NaN,1,Belgium,1979.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Portugal,2014.0
447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Portugal,2014.0
448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Portugal,2014.0
449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,Portugal,2014.0


In [43]:
treatment_country = 'United Kingdom'
control_countries = dat['country'].unique().tolist().remove(treatment_country)



times_dep = {'rr_votes': [1979, 1994]}
times_pred = {
    'E_eubadIP': [1979, 1994],
    'E_NatDem_satisfiedIP': [1979, 1994],
    'unempIP': [1979, 1994],
    'opencIP': [1979, 1994],
    'S_uegenIP': [1979, 1994],
    'rightgovIP': [1979, 1994],
    'ch_imp_fns': [1979, 1994],
    'eu_pos_meanIP': [1979, 1994],
    'rr_votes': [1994]
}

# calculate weights using Synth package
import SparseSC

SparseSC = synth.fit(
    predictors=list(times_pred.keys()),
    predictors_op='mean',
    time_predictors_prior=[1979, 1984, 1989, 1994],
    special_predictors=[('rr_votes', [1979, 1984, 1989, 1994], 'mean')],
    treatment_period=1999,
    controls=control_countries,
)

ModuleNotFoundError: No module named 'SparseSC'